In [ ]:
from gql import Client, gql
from gql.transport.requests import RequestsHTTPTransport
from phoenix.server.api.types.pagination import (
    cursor_to_id,
    cursor_to_offset,
    id_to_cursor,
    offset_to_cursor,
)

new_url = "http://127.0.0.1:6006/graphql"
old_url = "http://127.0.0.1:6008/graphql"

new_project_id = "UHJvamVjdDox"
old_project_id = "UHJvamVjdDow"

new_client = Client(
    transport=RequestsHTTPTransport(url=new_url, timeout=1),
    fetch_schema_from_transport=True,
)
old_client = Client(
    transport=RequestsHTTPTransport(url=old_url, timeout=1),
    fetch_schema_from_transport=True,
)

In [ ]:
spans_query = gql(
    """query SpansQuery($projectId: GlobalID!, $after: String = null, $before: String = null, $first: Int = null, $last: Int = null) {
  node(id: $projectId) {
    ...on Project {
      spans(after: $after, before: $before, first: $first, last: $last) {
        edges {
          cursor
        }
      }
    }
  }
}"""
)

In [ ]:
new_response = new_client.execute(
    spans_query, variable_values={"projectId": new_project_id, "first": 10}
)
old_response = old_client.execute(
    spans_query, variable_values={"projectId": old_project_id, "first": 10}
)
new_cursors = [edge["cursor"] for edge in new_response["node"]["spans"]["edges"]]
new_ids = [cursor_to_id(cursor) for cursor in new_cursors]
old_cursors = [edge["cursor"] for edge in old_response["node"]["spans"]["edges"]]
new_offsets = [cursor_to_offset(cursor) for cursor in old_cursors]
print(new_ids)
print(new_offsets)

In [ ]:
new_response = new_client.execute(
    spans_query,
    variable_values={
        "projectId": new_project_id,
        "after": id_to_cursor(10),
        "first": 10,
    },
)
old_response = old_client.execute(
    spans_query,
    variable_values={
        "projectId": old_project_id,
        "after": offset_to_cursor(10),
        "first": 10,
    },
)
new_cursors = [edge["cursor"] for edge in new_response["node"]["spans"]["edges"]]
new_ids = [cursor_to_id(cursor) for cursor in new_cursors]
old_cursors = [edge["cursor"] for edge in old_response["node"]["spans"]["edges"]]
new_offsets = [cursor_to_offset(cursor) for cursor in old_cursors]
print(new_ids)
print(new_offsets)